In [5]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import string
import pandas as pd
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [6]:
sources = pd.read_csv('datosencvesp.csv',sep=',')

sources


,Marca temporal,Fecha de nacimiento,Género,Nivel educacional,Comuna de residencia,1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada.,2. ¿Cómo describirías la diferencia entre la expresión C y D?\nC. Luis se gana a vender aliños en la feria los sábados.\nD. Luis se pone a vender aliños en la feria los sábados.,3. ¿Cómo describirías la diferencia entre la expresión E y F?\nE. Paula se echó a correr cuando sintió ese ruido.\nF. Paula se largó a correr cuando sintió ese ruido.,4. ¿Cómo describirías la diferencia entre la expresión G y H?\nG. José se puso a trabajar como mueblista.\nH. José se metió a trabajar como mueblista.,5. ¿Cómo describirías la diferencia entre la expresión I y J?\nI. Roberto partió a estudiar temprano hoy día.\nJ. Roberto comenzó a estudiar temprano hoy día.,...,"6. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [La niña se largó a tener sed]","6. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [La niña empezó a tener sed]","6. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [La niña rompió a tener sed]","6. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [La niña se echó a tener sed]","7. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [Se pusieron a trabajar mil quinientas personas]","7. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [Se largaron a trabajar mil quinientas personas]","7. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [Empezaron a trabajar mil quinientas personas]","7. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [Rompieron a trabajar mil quinientas personas]","7. Ordena las siguientes frases de tal forma que la primera (1) sea la que te parece más natural (o menos forzada) hasta llegar a la última (5), menos natural (o más forzada). [Se echaron a trabajar mil quinientas personas]","¿Hubo ejemplos que te causaron mucha extrañeza o ""sonaron raros""?¿Podrías señalar cuáles y explicar con tus propias palabras por qué?"
0,12/09/2022 16:29:14,13/05/2002,mujer,Universitaria incompleta,macul,sinonimas,sinonimas,sinonimas,por alguna razón siento que en la h se siente ...,en la expresión i es como si roberto se despla...,...,4,1,5,3,2,3,1,5,4,en general los ejemplos de la niña me sonaron ...
1,12/09/2022 18:17:17,29/08/1996,Masculino,Universitaria incompleta,Ñuñoa,Una marca el comienzo de la lluvia y la otra s...,"No veo diferencia, ambas apuntan literalmente ...","No veo diferencia, ambas se refieren a la mism...","No veo diferencias, para mí ambas se refieren ...",La diferencia se puede adjudicar a que en la e...,...,4,1,5,2,1,3,2,5,4,"""La niña se puso a tener sed"" me parece poco u..."
2,12/09/2022 19:53:00,17/10/1995,Masculino,Universitaria completa,Quilicura,La segundo tiene un pronombre clitico,La primera tiene un verbo que forma parte de u...,La segundo se más intensa,Se puso a trabajar tiene una connotacion más v...,La primera tiene un sema de movimiento asociado,...,2,1,5,4,1,3,2,4,5,Rompieron a trabajar
3,20/09/2022 

In [7]:
sources = sources[['1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada.', '2. ¿Cómo describirías la diferencia entre la expresión C y D?\nC. Luis se gana a vender aliños en la feria los sábados.\nD. Luis se pone a vender aliños en la feria los sábados.', '2. ¿Cómo describirías la diferencia entre la expresión C y D?\nC. Luis se gana a vender aliños en la feria los sábados.\nD. Luis se pone a vender aliños en la feria los sábados.', '4. ¿Cómo describirías la diferencia entre la expresión G y H?\nG. José se puso a trabajar como mueblista.\nH. José se metió a trabajar como mueblista.', '5. ¿Cómo describirías la diferencia entre la expresión I y J?\nI. Roberto partió a estudiar temprano hoy día.\nJ. Roberto comenzó a estudiar temprano hoy día.']]
sources.columns = ['1', '2', '3', '4', '5']
## selecciono solo las columnas qye usaré y las renombro

In [8]:
sources

,1,2,3,4,5
0,sinonimas,sinonimas,sinonimas,por alguna razón siento que en la h se siente ...,en la expresión i es como si roberto se despla...
1,Una marca el comienzo de la lluvia y la otra s...,"No veo diferencia, ambas apuntan literalmente ...","No veo diferencia, ambas apuntan literalmente ...","No veo diferencias, para mí ambas se refieren ...",La diferencia se puede adjudicar a que en la e...
2,La segundo tiene un pronombre clitico,La primera tiene un verbo que forma parte de u...,La primera tiene un verbo que forma parte de u...,Se puso a trabajar tiene una connotacion más v...,La primera tiene un sema de movimiento asociado
3,"Sinónimas, tal vez acotaría que el uso de ""se ...",Sinónimas,Sinónimas,El verbo metió me da a pensar más en el lugar ...,"""partió a"" me da a entender que el sujeto tuvo..."
4,"En el ejemplo A, entiendo que comienza a llove...",sinónimas,sinónimas,G: Entiendo que se ha puesto a trabajar como u...,I: Roberto salió (desplazamiento) a estudiar.\...
...,...,...,...,...,...
121,“ Se largó” es más fuerte que “empezó”,Se gana es un verbo mal usado,Se gana es un verbo mal usado,G es que empezó a hacerlo solo\nH que entró a ...,I que fue a un lugar a estudiar\nJ que empezó ...
122,Tendería a pensar que la lluvia mencionada en ...,Solo veo una diferencia de uso: en la oración ...,Solo veo una diferencia de uso: en la oración ...,Sinónimas,En la expresión 'I' podría tratarse de que Rob...
123,"La expresión A indica que comenzó a llover, si...",La expresión C indica que Luis se gana la vida...,La expresión C indica que Luis se gana la vida...,La expresión G indica que José trabaja como ma...,La expresión I indica que Roberto estaba hacie...
124,La opción B se siente como una expresión españ...,"Se entiende que significan lo mismo, pero la D...","Se entiende que significan lo mismo, pero la D...",La G podría dar a entender que está trabajando...,En la I tambiénbse podría dar a entender que s...


In [17]:
f = sources

In [20]:
f.list()

AttributeError: 'DataFrame' object has no attribute 'list'

# 1. ¿Cómo describirías la diferencia entre la expresión A y B? 
## A. Empezó a llover en la madrugada.
## B. Se largó a llover en la madrugada.

In [3]:
print(sources['1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada.'])
## señalo que trabajaré con esta columna

0                                              sinonimas
1      Una marca el comienzo de la lluvia y la otra s...
2                  La segundo tiene un pronombre clitico
3      Sinónimas, tal vez acotaría que el uso de "se ...
4      En el ejemplo A, entiendo que comienza a llove...
                             ...                        
121               “ Se largó” es más fuerte que “empezó”
122    Tendería a pensar que la lluvia mencionada en ...
123    La expresión A indica que comenzó a llover, si...
124    La opción B se siente como una expresión españ...
125                                           sinónimas 
Name: 1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada., Length: 126, dtype: object


In [4]:
a = (sources['1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada.'])

In [14]:
ab = (sources['1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada.'])

In [15]:
ab[:5]

0                                            sinonimas
1    Una marca el comienzo de la lluvia y la otra s...
2                La segundo tiene un pronombre clitico
3    Sinónimas, tal vez acotaría que el uso de "se ...
4    En el ejemplo A, entiendo que comienza a llove...
Name: 1. ¿Cómo describirías la diferencia entre la expresión A y B?\nA. Empezó a llover en la madrugada.\nB. Se largó a llover en la madrugada., dtype: object

In [5]:
a=[string.lower() for string in a]
## minúscula

In [6]:
a=[string.strip() for string in a]
## adios espacios en blanco extras

a[:10]

['sinonimas',
 'una marca el comienzo de la lluvia y la otra se refiere a la intensidad de esta.',
 'la segundo tiene un pronombre clitico',
 'sinónimas, tal vez acotaría que el uso de "se largó" implicaría más fuerza en la lluvia/la cantidad de lluvia.',
 'en el ejemplo a, entiendo que comienza a llover en la madrugada, es decir, no estaba lloviendo hace dos minutos, sin embargo, ahora si. en el ejemplo b, comprendo que estaba lloviendo previamente y comienza a llover con fuerza en la madrugada.',
 'a. la lluvia inició en la madrugada\nb. no entiendo su uso',
 'empezar es iniciar y se largo es que quedó la caga de una',
 'expresión b es más informal y asumo que la relación entre hablantes es más cercana.',
 'b es algo que usaria un extranjero en comparación a a',
 'empezó implica una lluvia normal, se largó implica un estuvo fuerte la lluvia']

In [7]:
len(a)

126

In [8]:
import networkx as nx
G = nx.Graph()